<center><img src="./assets/RAG-pipline.png" width=90%><p>使用 LangChain 完成蓝框中的 pipeline </p></center>


---

## 1. 文档解析 - PDF2Text

本次比赛提供的 PDF 文档中主要以 **文字** 和 **表格** 内容为主，这里使用在处理表格方面表现不错的 ``pdfplumber`` 包。

还可以尝试其他 PDF 处理工具，这些工具的效果各有千秋，下面给出一些参考：

- [LangChain v0.3 Documentation - How to load PDFs](https://python.langchain.com/docs/how_to/document_loader_pdf)

- [LangChain v0.2 Documentation (中文版) - 如何加载 PDF](https://python.langchain.com/docs/how_to/document_loader_pdf)

- [一文打尽PDF解析的工程实践——Langchain-RAG系列(1)](https://zhuanlan.zhihu.com/p/680175740)

此外，PDF 文档中可能存在一些图片信息，图片信息不会出现在解析结果中。不过图片中的内容可能用文字描述过，已经隐含在文本信息内。

In [1]:
import os
import pdfplumber

def parse_pdfs_from_folder(folder_path):
    all_texts = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".pdf"):
            print(f"Parsing {filename}")
            with pdfplumber.open(os.path.join(folder_path, filename)) as pdf:
                text = ""
                for page in pdf.pages:
                    text += page.extract_text()
                all_texts.append(text)
    return all_texts

folder_path = "./data/A_document"
texts = parse_pdfs_from_folder(folder_path)

Parsing AF33.pdf
Parsing AW18.pdf
Parsing AT17.pdf
Parsing AY12.pdf
Parsing AW10.pdf
Parsing AY05.pdf
Parsing AW32.pdf
Parsing AT02.pdf
Parsing AZ08.pdf
Parsing AZ01.pdf
Parsing AW13.pdf
Parsing AW05.pdf
Parsing AW24.pdf
Parsing AF37.pdf
Parsing AT09.pdf
Parsing AW08.pdf
Parsing AF16.pdf
Parsing AT13.pdf
Parsing AY02.pdf
Parsing AW23.pdf
Parsing AW28.pdf
Parsing AW03.pdf
Parsing AW07.pdf
Parsing AT21.pdf
Parsing AF31.pdf
Parsing AF02.pdf
Parsing AY07.pdf
Parsing AZ07.pdf
Parsing AY14.pdf
Parsing AT05.pdf
Parsing AW26.pdf
Parsing AY15.pdf
Parsing AZ04.pdf
Parsing AT07.pdf
Parsing AW12.pdf
Parsing AW09.pdf
Parsing AW06.pdf
Parsing AT25.pdf
Parsing AT24.pdf
Parsing AW04.pdf
Parsing AW34.pdf
Parsing AT32.pdf
Parsing AT19.pdf
Parsing AT23.pdf
Parsing AF29.pdf
Parsing AF20.pdf
Parsing AY11.pdf
Parsing AW33.pdf
Parsing AZ02.pdf
Parsing AZ05.pdf
Parsing AT29.pdf
Parsing AT03.pdf
Parsing AF01.pdf
Parsing AT27.pdf
Parsing AT34.pdf
Parsing AY08.pdf
Parsing AF11.pdf
Parsing AT14.pdf
Parsing AF26.p

查看部分 PDF 文档内容，并检查是否解析正确 (如文本顺序是否正确、表格内容是否正确等)

In [2]:
# texts[3] 对应 AY12.pdf
print(">>> 检查第 1 页标题:\n", texts[3][0:50])
print("\n>>> 检查第 2 页 2.2 表格:\n", texts[3][801:950])

>>> 检查第 1 页标题:
 中国联合网络通信股份有限公司 2019年半年度报告摘要
公司代码：600050 公司简称：中国联通


>>> 检查第 2 页 2.2 表格:
 2中国联合网络通信股份有限公司 2019年半年度报告摘要
2.2 公司主要财务数据
单位：元 币种：人民币
本报告期末
本报告期末 上年度末
比上年度末增减(%)
总资产 566,546,628,977 541,762,283,984 4.6
归属于上市公司股东的净资产 141,411,066,2


In [28]:
# texts[116] 对应 AZ06.pdf，里面包含一些图片内容
print(">>> 检查第 1 页标题:\n", texts[116][0:50])
print("\n>>> 检查第 17 页 (里面有一张图片，没有被解析):\n", texts[116][14400:14700])

>>> 检查第 1 页标题:
 No.202327
全球数字经济白皮书
(2023 年)
中国信息通信研究院
2024年1月版权声明

>>> 检查第 17 页 (里面有一张图片，没有被解析):
 字经济持续渗透。受行业属
性等因素影响，从全球看，数字技术在传统产业的应用率先在第三
产业爆发、数字化效果最显著，在第二产业的应用效果有待持续释
放，在第一产业的应用受到自然条件、土地资源等因素限制，仍需
探索更加适合的数字化解决方案。2022 年，全球 51 个主要经济体第
三、二、一产业数字经济增加值占行业增加值比重分别为 45.7%、
24.7%和 9.1%，分别较去年提升 0.7、0.5 和 0.2 个百分点。
来源：中国信息通信研究院
图22022年全球数字经济整体发展情况
（二）全球数字经济多极化趋势进一步深化
整体看，中、美、欧基于市场、技术、规则等方面优势，持续
加大数字经济发


In [7]:
# texts[101] 对应 AT11.pdf
print(">>> 检查第 1 页标题:\n", texts[101][0:140])
print("\n>>> 检查第 2 页最后的内容:\n", texts[101][3300:])

>>> 检查第 1 页标题:
 本文档为2024CCFBDCI比赛用语料的一部分。部分文档使用大语言模型改写生成，内容可能与现实情况
不符，可能不具备现实意义，仅允许在本次比赛中使用。
学思践悟担使命 走在前列建新功 ——中国联通党员干部热议深入开展主题教育感悟收获
发布时间：2023-07-31发布人：新闻宣

>>> 检查第 2 页最后的内容:
 动发展战略总体要求、
重要遵循和工作要求，努力探索实现金融与通信跨界创新成果落地，将数字人民币金融基础设施与信息技术基础设施相结合，
推动联通SIM卡硬钱包产品广泛应用于消费、交通、校园等场景，助力主业高质量发展。
万里征程风正劲，千钧重任再扬帆。中国联通将持续高质量开展主题教育，坚持学思用贯通、知信行统一，把习近平新时
代中国特色社会主义思想落实到改造主观世界和客观世界的实践中，切实把学习成果转化为坚定理想信念、落实战略规划和推
进中国联通高质量发展的强大力量。


**🚀 改进建议：**

在观察解析后的文档内容后，可以总结一些缺陷并进行优化，例如:

- 去掉 '\n'，使句子更加连续通顺 （如果 '\n' 前面是 '。'，可以保留）

- 去掉 '本文档为2024CCFBDCI比赛用语料的一部分。部分文档使用大语言模型改写生成，内容可能与现实情况\n不符，可能不具备现实意义，仅允许在本次比赛中使用。\n'

- 用 ``text += page.extract_text()`` 解析出的表格不够明确，可以尝试 ``table += page.extract_tables()`` 得到更加格式化的表格
    
    - 这样的表格可以独立于 `text`，单独保存到 `all_tables` 中
    
    - 此外，答案存在于文档表格中的问题对应的**难度系数分**更高，能更好的利用表格数据是一个提分点

- ......

## 2. 文本分块 (Chunk)

这里给出一种简单的实现方式，使用 ``LangChain`` 中推荐的 `RecursiveCharacterTextSplitter`，按顺序分隔。

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 分块函数
def chunk_texts(texts, chunk_size=500, chunk_overlap=100):
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    chunks = []
    for text in texts:
        chunks.extend(splitter.split_text(text))
    return chunks

chunks = chunk_texts(texts, chunk_size=300, chunk_overlap=100)
print(f"Number of chunks: {len(chunks)}")

Number of chunks: 4105


In [14]:
chunks[:3]

['本文档为2024CCFBDCI比赛用语料的一部分。部分文档使用大语言模型改写生成，内容可能与现实情况\n不符，可能不具备现实意义，仅允许在本次比赛中使用。\n中国联通发布经济社会智能孪生系统服务经济治理效能提升\n发布时间： 发布人：新闻宣传中心\n7 月 20 日，2024 中国联20通24合-0作7伙-2伴2大会人工智能赋能数据要素论坛在上海\n世博中心举办，论坛上中国联通智慧足迹总经理李振军与多位院士专家共同发布\n经济社会智能孪生系统。\n高层次专家云集，共同见证重要发布\n本次发布会邀请了多位学术界和行业界的顶尖专家参与。其中包括加拿大工',
 '世博中心举办，论坛上中国联通智慧足迹总经理李振军与多位院士专家共同发布\n经济社会智能孪生系统。\n高层次专家云集，共同见证重要发布\n本次发布会邀请了多位学术界和行业界的顶尖专家参与。其中包括加拿大工\n程院院士、加拿大工程研究院院士、欧洲科学院院士、郑州大学学术副校长杨天\n若，中国金融四十人论坛学术顾问许宪春，中国社会科学院国家未来城市实验室\n主任刘治，厦门大学经济学院、王亚南经济研究院院长周莹刚，国家发展和改革\n委员会经济与技术经济研究所室副主任杨威等重量级嘉宾。他们的到来不仅为本\n次发布会增添了学术和专业的光彩，更为经济社会智能孪生系统的应用和发展提\n供了坚实的理论支持和智力保障。',
 '委员会经济与技术经济研究所室副主任杨威等重量级嘉宾。他们的到来不仅为本\n次发布会增添了学术和专业的光彩，更为经济社会智能孪生系统的应用和发展提\n供了坚实的理论支持和智力保障。\n紧跟政策导向，服务国家战略需求\n在当前复杂多变的全球经济环境下，中国经济正面临着前所未有的挑战和机\n遇。二十届三中全会明确提出要在经济发展和社会民生等重点领域进一步深化改\n革，以应对全球化进程中的各种不确定因素。中国联通作为国家战略的重要参与\n者，始终将服务国家战略需求作为自身发展的核心使命。基于多年在数据和通信\n领域的积累与实践，中国联通智慧足迹推出的经济社会智能孪生系统，旨在通过']

**🚀 改进建议：**

- 对 `chunk_size` 和 `chunk_overlap` 进行调整

- 使用其他文本分块方法，可以参考：

    - [LangChain v0.3 Documentation - Text-Splitters](https://python.langchain.com/docs/how_to/#text-splitters)

    - [[A tutorial on Github] 5 Levels Of Text Splitting](https://github.com/FullStackRetrieval-com/RetrievalTutorials/blob/main/tutorials/LevelsOfTextSplitting/5_Levels_Of_Text_Splitting.ipynb)

- ......

## 3. 用 向量模型 对每个 Chunk 进行向量化处理，并存储到向量数据库

首先下载**比赛规定**的向量模型（Embedding model）: [bge-large-zh-v1.5](https://www.modelscope.cn/models/AI-ModelScope/bge-large-zh-v1.5)

- 下载时间 ~ 4 min

- 存储空间 ~ 2.5G

In [49]:
from modelscope import snapshot_download

to_your_path = '/data/lujd/modelscope/hub' 
embed_model_dir = snapshot_download('AI-ModelScope/bge-large-zh-v1.5', revision='master',
                                cache_dir=to_your_path)      # cache_dir 默认是 '~/.cache/modelscope/hub'
print(embed_model_dir)

2024-10-01 17:40:51,399 - modelscope - WARNING - Using branch: master as version is unstable, use with caution


/data/lujd/modelscope/hub/AI-ModelScope/bge-large-zh-v1___5


然后使用 ``langchain`` 中的 ``HuggingFaceBgeEmbeddings`` 加载 `bge-large-zh-v1.5` 模型

- 还有其他加载模型的方式，参考 [bge-large-zh-v1.5 - Usage](https://www.modelscope.cn/models/AI-ModelScope/bge-large-zh-v1.5#Usage)

- 如果加载到 GPU 上，占显存空间 ~ 1800M

In [17]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

embed_model_name = embed_model_dir
embed_model_kwargs = {'device': 'cuda:4'}           # 'cuda'/'cuda:4' for gpu
encode_kwargs = {'normalize_embeddings': True}      # set True to compute cosine similarity
embed_model = HuggingFaceBgeEmbeddings(
    model_name=embed_model_name,
    model_kwargs=embed_model_kwargs,
    encode_kwargs=encode_kwargs,
    query_instruction="为这个句子生成表示以用于检索相关文章："
)
print(embed_model)

client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': True}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
) model_name='/data/lujd/modelscope/hub/AI-ModelScope/bge-large-zh-v1___5' cache_folder=None model_kwargs={'device': 'cuda:4'} encode_kwargs={'normalize_embeddings': True} query_instruction='为这个句子生成表示以用于检索相关文章：' embed_instruction='' show_progress=False


测试**向量模型**

- 输入 `submit_example.csv` 中提供的 `answer`，检查输出向量与文件中提供的 `embedding` 是否一致

    - 也就是看是否正确使用了主办方规定的向量模型

- 跑 5 个 chunk (chunk_size=300): ~ 2000M（GPU）

In [18]:
submit_example_answer1 = ["我们坚定践行网络强国、数字中国、智慧社会战略部署，今天的中国联通，正在从传统运营商加速向数字科技领军企业转变，实现了四个维度的转型升级：一是联接规模和联接结构升维，从过去的连接人为主拓展到连接人机物，大力发展物联网和工业互联网；二是核心功能升维，从以基础连接为主发展到大联接、大计算、大数据、大应用、大安全五大主责主业；三是服务和赋能水平升维，以5G、云计算、大数据、人工智能、区块链为代表的新一代信息技术和实体经济的结合，服务数字政府、数字社会、数字经济的能力不断增强；四是发展理念升维，我们以传统的市场驱动为主转变为市场驱动和创新驱动相结合的发展模式，尤其是加大了科技创新及人才方面的投入力度，创新发展的动能得到了空前的释放。"]
embeddings = embed_model.embed_documents(submit_example_answer1)
print(len(embeddings), len(embeddings[0]), embeddings[0][:10])

# 与 submit_example.csv 中第 1 个 问题的 answer embedding 是一致的

1 1024 [-0.027079833671450615, -0.009818877093493938, -0.01249589491635561, -0.003817810909822583, 0.017168041318655014, 0.027743525803089142, 0.03685056418180466, -0.0333469994366169, -0.040052466094493866, 0.028761744499206543]


In [19]:
embeddings = embed_model.embed_documents(chunks[:5])
len(embeddings), len(embeddings[0])

(5, 1024)

对所有 chunks 进行向量化处理，并存储到向量数据库 （``FAISS``）

- 占显存空间 ~ 2600M

In [20]:
from langchain.docstore.document import Document
from langchain.vectorstores import FAISS

# 为每个 chunk 添加一个 doc_id，方便查找。
documents = [Document(page_content=chunk, metadata={'doc_id': i}) for i, chunk in enumerate(chunks)]
"""
还可以添加更多信息用于后面的检索策略，例如属于哪个文档、文档标题等。
"""

vector_store = FAISS.from_documents(documents, embed_model)
print(f"Number of stored vectors: {vector_store.index.ntotal}")

Number of stored vectors: 4105


## 4. 根据问题检索答案（文本召回）

### 4.1. 检索与问题关联度最高的文本

计算问题的向量，然后在向量数据库中进行相似度计算并排序，返回前 K 个（Top-K）最相似的文本块

In [21]:
def retrieve_docs(query, vector_store, model, top_k=5):
    query_embedding = model.embed_query(query)
    retrieved_docs = vector_store.similarity_search_by_vector(query_embedding, top_k)
    return retrieved_docs

import pandas as pd
submit_df = pd.read_csv('./data/A_question.csv', sep=',')

quesid2docs_dict = {}
for ind, row in submit_df.iterrows():
    quesid2docs_dict[row['ques_id']] = retrieve_docs(row['question'], vector_store, embed_model, top_k=5)

观察最后一个问题以及检索到的文本，会发现一些可以改进的点：

- “本文档为2024CCFBDCI比赛用语料的一部分。部分文档使用大语言模型改写生成，....” 这段与文档内容无关的话出现在 Top-1 文本中

- 排名第 3 的文本给出了正确答案：“.....分别是中国银行、国航、伊利集团、安踏和中国联通。” 而 Top-1 没有

In [22]:
row['question']             # 最后一个问题

'截至2017年12月26日，北京市2022年冬奥会和冬残奥会官方合作伙伴已达五家，分别是哪几家？'

In [23]:
quesid2docs_dict[100]       # 最后一个问题检索到的文本

[Document(metadata={'doc_id': 2519}, page_content='本文档为2024CCFBDCI比赛用语料的一部分。部分文档使用大语言模型改写生成，内容可能与现实情况\n不符，可能不具备现实意义，仅允许在本次比赛中使用。\n中国联通成为北京2022年冬奥会和冬残奥会官方通信服务合作伙伴\n发布时间：2017-12-26发布人：新闻发布人\n12月26日晚间，在中国联通大厦里，中国联通的红色“中国结”与北京2022年冬奥会会徽和冬残奥会会徽的组合标志格外\n醒目，中国联通与北京2022年冬奥会和冬残奥会组织委员会签约仪式在这里举行。这场签约，标志着中国联通正式成为北京\n2022年冬奥会和冬残奥会官方通信服务合作伙伴，跻身于北京冬奥组委市场开发计划最高级别的赞助企业之列。'),
 Document(metadata={'doc_id': 2520}, page_content='2022年冬奥会和冬残奥会官方通信服务合作伙伴，跻身于北京冬奥组委市场开发计划最高级别的赞助企业之列。\n当晚，北京市副市长、北京冬奥组委执行副主席张建东，中国残联理事长、北京冬奥组委副主席鲁勇；中国联通董事长王\n晓初，中国联通总经理陆益民，中国联通副总经理邵广禄出席签约仪式。北京冬奥组委秘书长韩子荣与中国联通副总经理买彦\n州代表双方签署赞助协议。\n北京市副市长、北京冬奥组委执行副主席张建东致辞表示，北京冬奥会是我国重要历史节点的重大标志性活动，对于展现\n新时代国家形象、促进国家发展、振奋民族精神，具有重大而深远的意义。与通信服务企业开展合作，不仅是实施市场开发计'),
 Document(metadata={'doc_id': 2523}, page_content='致的速率支撑奥运，以智慧的应用、丰富的产品服务奥运，以智慧的技术、专业的队伍保障奥运，为冬奥会的成功举办、为中\n国力量的再次彰显，注入强劲新动能。\n在签约仪式上，中国联通正式宣布启用166新号段，在全国投入百万个号码，以优质的网络、产品与服务持续助力中国通\n信事业和中国奥运事业。\n据了解，北京2022年冬奥会和冬残奥会市场开发计划自今年2月份启动以来，得到了社会各界的广泛关注和广大企业的踊\n跃参与。目前，北京2022年冬奥会和冬残奥会官方合作伙伴已达五家，分别是中国银行、国

**🚀 改进建议：**

- 使用其他的相似性指标

- 使用其他的检索策略，例如先过滤再检索、EnsembleRetriever 等，参考：[LangChain v0.3 Documentation - Retrievers](https://python.langchain.com/docs/how_to/#retrievers)

- 检索到 Top-K 文本后，再使用其他指标/模型/方法对这 Top-K 文本进行排序 （重排）

- ......

### 4.2. 如何将检索的文本变为问题的答案？

这里直接用贪心策略将检索到的 Top-1 文本作为答案，并给出相应的向量

In [24]:
def get_answer_and_embedding(retrieved_docs):
    # for example, select top1 retrieved_docs as the final answer
    top1_doc = retrieved_docs[0]

    # 文本内容
    answer = top1_doc.page_content

    # 根据前面设置的 doc_id 获取文本对应的向量
    doc_id = top1_doc.metadata['doc_id']
    embedding = vector_store.index.reconstruct(doc_id)

    return answer, embedding

quesid2answer_dict, quesid2embedding_dict = {}, {}
for quesid, retrieved_docs in quesid2docs_dict.items():
    answer, embedding = get_answer_and_embedding(retrieved_docs)
    quesid2answer_dict[quesid] = answer                 # str
    quesid2embedding_dict[quesid] = embedding           # np.array

**🚀 改进建议：**

- 将检索到的 Top-K 文本拼起来作为答案（但要保证不能太长，防止被惩罚），有如下考虑：

    - Top-1 未必是完整答案，可能是 Top-1 和 Top-2 拼起来。

    - 一些难度系数为 2 的问题，答案涉及“文档多个段落”或者“多个文档”，需要将多个文本块合并后返回。

- ......

- 如果想用 LLM 且条件允许，可以：
    
    - 将检索到的 Top-K 文本和 Query 写入 LLM 的 prompt template 中，然后输入到 LLM 中，推理得到一个更加综合自然的回答。但是比赛要求禁止使用 LLM 直接生成最终答案。

    - 所以用 LLM 的回答作为 reference，计算其与 Top-K 文本的相似度，然后将 Top-K 文本重新排序（重排），最后将重新排序后的 Top-1 作为答案。（**可能存在犯规风险**）

## 保存结果

将召回的答案及向量写入 `submit_df`，并保存到 `./submission/` 下

In [26]:
REMOVE_NEWLINE = True

if REMOVE_NEWLINE:      # 避免对方测试时使用其他读取方式，而导致读取错误
    submit_df['answer'] = submit_df['ques_id'].apply(lambda x: quesid2answer_dict[x].replace("\n", ""))
else:
    submit_df['answer'] = submit_df['ques_id'].apply(lambda x: quesid2answer_dict[x])

# embedding 注意与 submit_example.csv 的格式一致
submit_df['embedding'] = submit_df['ques_id'].apply(lambda x: str(list(quesid2embedding_dict[x]))[1:-1])

submit_df.head(2)

,ques_id,question,answer,embedding
0,1,根据年度报告，2022年中国联通在向数字科技领军企业转变的过程中实现了哪些维度的转型升级？,数字经济的不断深入发展，中国联通将继续秉持创新驱动的发展理念，不断提升数据管理能力和智能化水...,"-0.011659057, -0.016988184, -0.040713284, -0.0..."
1,2,告诉我2022年联通产业互联网收入的同比增长速度。,家庭产品，促进相互拉动发展。受激烈竞争影响，2019 年上半年，公司固网宽带接入收入同比下降...,"-0.009771062, -0.041889433, -0.022890702, -0.0..."


In [222]:
import datetime

# 可以按日期-时间保存，例如 "2024-10-01 21:28" 记为 "1001_2128"
today = datetime.datetime.now().strftime("%m%d_%H%M")
submit_savepath = f'./submission/submit_{today}.csv'
submit_df.to_csv(submit_savepath, sep=',', index=False)
print(f"Saved path: {submit_savepath}")

Saved path: ./submission/submit_1001_2128.csv
